In [1]:
########## SET-UP ##########

# necessary libraries
import os
import json
import requests
import time
import datetime

# INSERT YOUR API KEY HERE
KEY = 'b00c992b-6fc1-4795-abcc-7db85c1b72fa'


In [2]:
# MATCH-v2.2
REGION_ENDPOINT = "https://{0}.api.pvp.net/api/lol/{0}/"

# FUNCTION TO PULL MATCH DATA
def get_match(REGION, matchId, includeTimeline):
    """
    Retrieve match by match ID.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.2/match/{1}?"
         "api_key={2}&includeTimeline={3}").
        format(REGION, matchId, KEY, includeTimeline))

In [4]:
################## UNCOMMENT AND DELETE IRRELEVANT PARTS HERE ###############
################## DO NOT RUN THIS AGAIN #################

# List of starting MatchID's
# sean2029070300
match_id_list = [2029070300, 2028998508, 2028697992, 2028410883, 2028610390]
# jaemin
#match_id_list = [2028886202, 2028875395, 2028669368, 2028646565, 2028611382, 2026627159]
# lenny kei
#match_id_list = [2028144696, 2028076924, 2028017860, 2028129100, 2028115585, 2026610783]
# a paek
#match_id_list = [2028089458, 2027868106, 2027853644, 2027828374, 2026660053, 2026561470]

In [11]:
# Constants
# REGION = 'na'
STATUS_OK = 200
STATUS_RATE_LIMIT_EXCEEDED = 429
valid_rank_games = ['RANKED_SOLO_5x5',  # Ranked Solo 5v5 games
                    'RANKED_PREMADE_5x5',  # Ranked Premade 5v5 games
                    'RANKED_TEAM_5x5']  # Ranked Team 5v5 games

REGION_list = ['na', 'kr', 'eune', 'euw', 'lan', 'las', 'oce', 'ru', 'br', 'tr']

regional_match_id = {'na': 2028998508, 
                     'kr': 2193799029,   #캬하하
                     'br': 647354574,    #brTT
                     'eune': 1297787579, #PaiNkiller Gr3k
                     'euw': 2418259276,  #Arceo
                     'lan': 197101244,   #Malvado
                     'las': 272915603,   #Tea
                     'oce': 113859146,   #Talk Sleek
                     'ru': 84480380,     #bronyaw
                     'tr': 325233355}    #kua

# jaemin
regional_match_id = {'na': 2029960687,
                     'kr': 2192916925,
                     'br': 664106032,
                     'eune': 1293871277,
                     'euw': 2417718483,
                     'lan': 229932736,
                     'las': 271679926,
                     'oce': 113897495,
                     'ru': 83927963,
                     'tr': 332060998}
                     
# kenny
regional_match_id = {'na': 2029894256
                     'kr': 2181887785
                     'br': 664332857
                     'eune': 1293801833
                     'euw': 2417655730
                     'lan': 229928986
                     'las': 272582143
                     'oce': 113887342
                     'ru': 83706280
                     'tr': 332046148}

# andrew
regional_match_id = {'na': 2029848145
                     'kr': 2181825342
                     'br': 664520626
                     'eune': 1293842842
                     'euw': 2417484267
                     'lan': 229859020
                     'las': 272319645
                     'oce': 113877802
                     'ru': 84139484
                     'tr': 331322838}

curr_region = REGION_list[0]
current_match_id = regional_match_id[curr_region]

print curr_region
print current_match_id

na
2028998508


In [26]:
#rate limit reached at kr, moving onto eune
#rate limit reached at eune, moving onto euw
#rate limit reached at euw, moving onto lan
#rate limit reached at lan, moving onto las

get_match('oce', 113859146, True).json()

{u'mapId': 11,
 u'matchCreation': 1449052387998,
 u'matchDuration': 2186,
 u'matchId': 113859146,
 u'matchMode': u'CLASSIC',
 u'matchType': u'MATCHED_GAME',
 u'matchVersion': u'5.23.0.239',
 u'participantIdentities': [{u'participantId': 1,
   u'player': {u'matchHistoryUri': u'/v1/stats/player_history/NA/32100127',
    u'profileIcon': 968,
    u'summonerId': 890352,
    u'summonerName': u'Easy'}},
  {u'participantId': 2,
   u'player': {u'matchHistoryUri': u'/v1/stats/player_history/NA/37546743',
    u'profileIcon': 936,
    u'summonerId': 326444,
    u'summonerName': u'Huey Dewey Louie'}},
  {u'participantId': 3,
   u'player': {u'matchHistoryUri': u'/v1/stats/player_history/NA1/207117053',
    u'profileIcon': 936,
    u'summonerId': 288237,
    u'summonerName': u'SNT Zeraph'}},
  {u'participantId': 4,
   u'player': {u'matchHistoryUri': u'/v1/stats/player_history/OC1/200754077',
    u'profileIcon': 835,
    u'summonerId': 1505382,
    u'summonerName': u'I LOVE CHIEFS'}},
  {u'participant

In [ ]:
# Starting MatchID
# STARTING_MATCH_ID = 2026937783

# Functions that check status of the data pulls
def isValid(match):
    if match.status_code != STATUS_OK:
        return False
    return True

def isClassicMatch(match):
    j = match.json()
    if j['mapId'] == 11 and j['matchMode'] == "CLASSIC" and j['matchType'] == 'MATCHED_GAME':
        return True
    else:
        return False

def isRankMatch(match):
    j = match.json()
    if j['queueType'] in valid_rank_games:
        return True
    else:
        return False
                    
def rateLimitExceeded(match):
    if int(match.status_code) == STATUS_RATE_LIMIT_EXCEEDED:
        return True
    return False

# index for the big loop
i = 0
# counting the number of data retrieved
n = 0
# counting how many times we sleep in case we hit a dead-end
sleep_counter = 0
# region list index
r = 0

for k in range(100000):
    # break out of the loop when there are no more match_id's left
    #if match_id_list == []:
    #    print "ran out of starting match ID's. ending the loop"
    #    break
    
    # incrementing index
    #print ("i: %d, match_id: %d") % (i, match_id_list[0] + i)    
    
    # current region
    curr_region = REGION_list[r]
    current_match_id = regional_match_id[curr_region] + i
    i += 1
    
    m = get_match(curr_region, current_match_id, False)
    
    if not rateLimitExceeded(m):
        #print "rate limit not exceeded at %s" % curr_region
        if isValid(m) and isClassicMatch(m) and isRankMatch(m):
            f = open("dump_global/" + str(current_match_id)+".json", 'w+')        
            f.write(m.text.encode('utf-8'))
            n += 1
            print 'n: %d, ts: %s, ID: %d, date: %s, season: %s, tier: %s, loop_index: %d, region: %s' % (n, datetime.datetime.fromtimestamp(time.time()).strftime('%H:%M:%S'), 
                                                                                                 current_match_id, 
                                                                                                 ('%s' % (time.strftime('%m-%d', time.localtime(m.json()['matchCreation']/1000)))),
                                                                                                 m.json()['season'], m.json()['participants'][1]['highestAchievedSeasonTier'], k,
                                                                                                            m.json()['region'])
            f.close()
            # reset sleep_counter if we are get data again
            #sleep_counter = 0
    else:
        regional_match_id[curr_region] =  current_match_id + i
        if (r == 9):
            print ("rate limit reached at %s, moving onto %s") % (REGION_list[r], REGION_list[0])
            r = 0
        else:
            print ("rate limit reached at %s, moving onto %s") % (REGION_list[r], REGION_list[r+1])
            r += 1
        i = 0
        #time.sleep()
        
        #sleep_counter += 1
        #print("API rate limit reached - waiting 10 seconds, sleep count: %d, loop_index: %d") % (sleep_counter, i)
        #time.sleep(10)
        
        # if we have to sleep 30 times, we will assume that we are in no man's land and move onto the next id on list
        #if sleep_counter >= 30:
        #    del match_id_list[0]
        #    sleep_counter = 0
        #    i = 0
        #    print ("Seems like we hit a dead-end. Starting from the next match id on the list. new_starting_point: %d") % match_id_list[0]

rate limit reached at na, moving onto kr
rate limit reached at kr, moving onto eune
n: 1, ts: 16:36:41, ID: 1296485272, date: 12-04, season: PRESEASON2016, tier: PLATINUM, loop_index: 27, region: EUNE
n: 2, ts: 16:36:42, ID: 1296485273, date: 12-04, season: PRESEASON2016, tier: BRONZE, loop_index: 28, region: EUNE
n: 3, ts: 16:36:42, ID: 1296485274, date: 12-04, season: PRESEASON2016, tier: GOLD, loop_index: 29, region: EUNE
n: 4, ts: 16:36:43, ID: 1296485276, date: 12-04, season: PRESEASON2016, tier: SILVER, loop_index: 31, region: EUNE
rate limit reached at eune, moving onto euw
n: 5, ts: 16:36:45, ID: 2418260102, date: 12-06, season: PRESEASON2016, tier: SILVER, loop_index: 34, region: EUW
n: 6, ts: 16:36:47, ID: 2418260105, date: 12-06, season: PRESEASON2016, tier: GOLD, loop_index: 37, region: EUW
rate limit reached at euw, moving onto lan
n: 7, ts: 16:36:51, ID: 229585557, date: 12-04, season: PRESEASON2016, tier: SILVER, loop_index: 45, region: LAN
rate limit reached at lan, mov

In [28]:
valid_rank_games = ['RANKED_SOLO_5x5',  # Ranked Solo 5v5 games
                    'RANKED_PREMADE_5x5',  # Ranked Premade 5v5 games
                    'RANKED_TEAM_5x5']  # Ranked Team 5v5 games

In [36]:
regional_match_id['br'] = 665053482

In [ ]:
664520626